In [1]:
# Carga de librerías
import numpy as np
import tensorflow as tf
import PIL
import PIL.Image
from PIL import Image
import pathlib
import scipy as sp
import seaborn as sns
from itertools import product
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import time
import torch
import scipy.spatial
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import os

In [2]:
import pickle
import sys
from pathlib import Path

# Añadir el directorio raíz del proyecto a sys.path
module_path = Path('/Users/Cesar/Desktop/Proyecto-CSG/').resolve()
if module_path not in sys.path:
    sys.path.append(str(module_path))

# Añadir el directorio que contiene 'new_spectral_metric' a sys.path
new_spectral_metric_path = module_path / '/Users/Cesar/Desktop/Proyecto-CSG/pytorch_spectral_metric'
if new_spectral_metric_path not in sys.path:
    sys.path.append(str(new_spectral_metric_path))

from numpy.linalg import LinAlgError
from scipy.sparse.csgraph import laplacian
from pytorch_spectral_metric.estimator_pytorch import CumulativeGradientEstimator
from pytorch_spectral_metric.visualize_pytorch import make_graph
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Conv2D, Dense, MaxPooling2D, Lambda
from tensorflow.keras import backend as K
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib.collections import LineCollection
from matplotlib.font_manager import FontProperties
from pathlib import Path



In [3]:
print("GPU disponible:", torch.cuda.is_available())
print("Nombre de la GPU:", torch.cuda.get_device_name() if torch.cuda.is_available() else "N/A")


GPU disponible: True
Nombre de la GPU: NVIDIA GeForce RTX 4060


In [2]:

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision import datasets

# Definir directorios
directorio_dataset = '../Datasets/CIFAR-10 dos clases'
imagenes_originales = '../Datasets/CIFAR-10 dos clases/cat'
imagenes_corruptas = '../Datasets/CIFAR-10 dos clases/cat_c_defocus_blur'

directorio_dataset = pathlib.Path(directorio_dataset)
imagenes_originales = pathlib.Path(imagenes_originales)
imagenes_corruptas = pathlib.Path(imagenes_corruptas)



# Dimensiones de imagen y tamaño de batch
img_height = 32
img_width = 32
#batch_size = len(list(data_dir.glob('*/*.png'))) # leer todas las imágenes al tiempo


# Transformaciones para preprocesamiento de datos
transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),  # Ajustar tamaño a 32x32
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizar
])

class CustomImageDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = pathlib.Path(directory)
        self.transform = transform
        self.images = list(self.directory.glob('*.png'))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image



# Crear datasets personalizados
dataset_completo = datasets.ImageFolder(root=directorio_dataset, transform=transform)
dataset_originales = CustomImageDataset(imagenes_originales, transform=transform)
dataset_corruptas = CustomImageDataset(imagenes_corruptas, transform=transform)

# Calcular totales
total_dataset = len(dataset_originales) + len(dataset_corruptas)
total_originales = len(dataset_originales)
total_corruptas = len(dataset_corruptas)

batch_size = total_dataset



# Crear el DataLoader para iterar sobre los datos en lotes
# DataLoader para cargar los datasets
dataloader_completo = DataLoader(dataset_completo, batch_size=batch_size, shuffle=False, num_workers=0)
dataloader_originales = DataLoader(dataset_originales, batch_size=batch_size, shuffle=False)
dataloader_corruptas = DataLoader(dataset_corruptas, batch_size=batch_size, shuffle=False)


print(f"Total imágenes dataset: {total_dataset}")
print(f"Total imágenes originales: {total_originales}")
print(f"Total imágenes corruptas: {total_corruptas}")




NameError: name 'pathlib' is not defined

In [1]:
# Acceder a las clases

class_names = dataset_completo.classes
num_classes = len(class_names)

print(f"Número de clases: {num_classes}")
print(f"Los nombres de las clases son: {class_names}")


NameError: name 'dataset_completo' is not defined

In [6]:
# Iterar sobre los datos para verificar las etiquetas
for images, labels in dataloader_completo:
    labels = labels.numpy() if hasattr(labels, 'numpy') else labels
    for label in labels:
        assert label in [0, 1], f"Etiqueta inesperada {label} en la imagen."

print("Verificación de etiquetas exitosa.")

Verificación de etiquetas exitosa.


In [7]:
# Iterar sobre los datos para obtener el primer lote
for images, labels in dataloader_completo:
    numpy_all_images = images.numpy()  # Convertir el tensor de imágenes a numpy
    numpy_all_labels = labels.numpy()  # Convertir el tensor de etiquetas a numpy
    
    # Escalar las imágenes al rango [0, 1]
    numpy_all_images /= 255.0
    
    print(numpy_all_images.shape)
    break  # Solo tomamos el primer lote para imprimir las dimensiones

(10000, 3, 32, 32)


In [8]:
fully_dataset = torch.tensor(numpy_all_images.reshape((numpy_all_images.shape[0], -1)), dtype=torch.float32)  
fully_labels = torch.tensor(numpy_all_labels, dtype=torch.int32)  

estimator = CumulativeGradientEstimator(M_sample=5000, k_nearest=10)  
estimator.fit(data=fully_dataset, target=fully_labels)  
csg = estimator.csg  

make_graph(estimator.difference, title="Comparative between two clases, CSG: "+str(csg), classes=class_names)  

KeyboardInterrupt: 

In [ ]:
pairs = list(zip(*np.unravel_index(torch.argsort(estimator.W, axis=None), estimator.W.shape)))  
pairs = [(original, corrupto) for original, corrupto in pairs if original != corrupto]  

similitud_de_clases = pairs[0]  
original, corrupto = similitud_de_clases  

print("Calculo de similitud entre clases")  
lst = []  
for idx, (original, corrupto) in enumerate(pairs[::2][:1]):  
    lst.append({"Clases a comparar" : f"{class_names[original]} <> {class_names[corrupto]}", "Similitud": estimator.W[original, corrupto].item()})  
print(pd.DataFrame(lst))  

In [ ]:
filenames = [str(f) for f in directorio_dataset.glob('*/*.png')]  
filenames_dict = {i: f.name for i, f in enumerate(directorio_dataset.glob('*/*.png'))}  

for class_name, indices in estimator.class_indices.items():  
    print(f"Clase: {class_names}, Número de muestras: {len(indices)}")  

In [ ]:
def show_least_similar_using_P(source_intent, target_intent, estimator, class_names, filenames_dict):  
    print(f"{class_names[source_intent]} <> {class_names[target_intent]}")  
    
    P = estimator.P[(source_intent, target_intent)]  
    least_similar_indices = torch.argsort(P, axis=None)  
    
    shown_files = set()  
    count = 0  
    
    for idx in least_similar_indices:  
        if count >= 10:  
            break  
        
        idx_2d = torch.unravel_index(idx, P.shape)  
        
        source_idx = estimator.class_indices[source_intent][idx_2d[0]]  
        target_idx = estimator.class_indices[target_intent][idx_2d[1]]  
        
        if source_idx < len(filenames_dict) and target_idx < len(filenames_dict):  
            source_filename = filenames_dict[int(source_idx)]  
            target_filename = filenames_dict[int(target_idx)]  
            
            if source_filename not in shown_files and target_filename not in shown_files:  
                shown_files.add(source_filename)  
                shown_files.add(target_filename)  
                
                value = P[idx_2d]  
                print(f"\tNombre del archivo: {source_filename}, Clase: {class_names[source_intent]}, Similitud: {value:.4f}")  
                count += 1  

pairs = [(0, 1)]  
first_pair = pairs[0]  

show_least_similar_using_P(first_pair[0], first_pair[1], estimator, class_names, filenames_dict)  
show_least_similar_using_P(first_pair[1], first_pair[0], estimator, class_names, filenames_dict)  

for class_name, indices in estimator.class_indices.items():  
    print(f"Clase: {class_name}, Número de muestras: {len(indices)}")  

In [ ]:
# Función para mover los datos a la GPU si está disponible
def to_device(tensor):
    if torch.cuda.is_available():
        return tensor.to('cuda')
    return tensor

# Aplanar las imágenes
fully_dataset = numpy_all_images.reshape((numpy_all_images.shape[0], numpy_all_images.shape[1]*numpy_all_images.shape[2]*numpy_all_images.shape[3])) # es un arreglo bidimensional donde cada fila representa una imagen y cada columna representa un píxel de la imagen
fully_labels = numpy_all_labels.reshape(numpy_all_labels.shape[0],)

# Convertir los datos a tensores de PyTorch y mover a la GPU si es posible
fully_dataset = to_device(torch.tensor(fully_dataset, dtype=torch.float32))
fully_labels = to_device(torch.tensor(fully_labels, dtype=torch.int64))

# Inicializar el estimador
print("Inicializando el estimador...")
estimator = CumulativeGradientEstimator(M_sample=1000, k_nearest=10)

print("Comenzando a ajustar el estimador...")
start_time = time.time()
estimator.fit(data=fully_dataset, target=fully_labels)
end_time = time.time()
print(f"Tiempo de ajuste del estimador: {end_time - start_time} segundos")

print("Calculando csg...")
csg = estimator.csg  # The actual complexity values.
print(f"csg: {csg}")

print("Obteniendo evals y evecs...")
print(f"Eigenvalues: {estimator.evals}")
print(f"Eigenvectors: {estimator.evecs}")

print("Creando el gráfico...")
make_graph(estimator.difference, title="Comparative between two clases, CSG: "+str(csg), classes=class_names)
print("Gráfico creado.")



Inicializando el estimador...
Comenzando a ajustar el estimador...
Inicio de fit...
Number of classes (n_class): 2
Llamando a find_samples...
Llamando a compute...
similarity_arrays[0][0] = SimilarityArrays(sample_probability=tensor([5.9404e-01, 1.2318e-04, 2.2524e-01, 2.4316e-01, 6.8169e-01, 3.6394e-01,
        9.3357e-01, 7.8260e-01, 5.8198e-01, 3.6618e-02, 5.9366e-01, 3.3656e-01,
        5.8335e-01, 4.9149e-01, 4.0820e-01, 8.5336e-01, 7.1988e-01, 1.9224e-01,
        9.9240e-01, 3.2394e-02, 5.9394e-01, 7.1521e-01, 1.9339e-01, 9.2015e-01,
        2.9382e-01, 9.7192e-01, 9.5258e-01, 6.5302e-01, 1.5766e-01, 3.0356e-02,
        9.8507e-01, 8.8734e-01, 9.7378e-01, 6.9589e-01, 4.4610e-01, 7.8617e-01,
        9.4821e-01, 1.6777e-01, 7.1574e-02, 5.2886e-01, 8.5242e-01, 9.0038e-02,
        6.2327e-01, 6.8098e-01, 3.9014e-01, 1.5634e-01, 8.6221e-01, 5.0164e-01,
        5.5675e-01, 3.5495e-01, 3.8965e-01, 4.2844e-02, 7.7835e-01, 6.7383e-01,
        6.2363e-01, 9.0523e-01, 5.4150e-01, 5.2820e-01

c:\Users\Cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\spatial\distance.py:1129: RuntimeWarning: invalid value encountered in scalar divide
  return l1_diff.sum() / l1_sum.sum()


_LinAlgError: linalg.eigh: The algorithm failed to converge because the input matrix is ill-conditioned or has too many repeated eigenvalues (error code: 1).

In [ ]:
# Ordena los pares de clases en orden de similitud ascendente (menos similar primero)
pairs = list(zip(*np.unravel_index(np.argsort(estimator.W, axis=None), estimator.W.shape)))
pairs = [(original,corrupto) for original,corrupto in pairs if original != corrupto]

similitud_de_clases = pairs[0]
original, corrupto = similitud_de_clases  # Pares de clases menos similares

print("Calculo de similitud entre clases")
lst = []
for idx, (original,corrupto) in enumerate(pairs[::2][:1]):
    lst.append({"Clases a comparar" : f"{class_names[original]} <> {class_names[corrupto]}", "Similitud": estimator.W[original,corrupto]})
print(pd.DataFrame(lst))
print(estimator.W.shape)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:

# Obtener todos los nombres de archivo en el dataset
filenames = [str(f) for f in directorio_dataset.glob('*/*.png')]

# Crear un mapeo de índices a nombres de archivo (solo el nombre de archivo, sin la ruta)
filenames_dict = {i: f.name for i, f in enumerate(directorio_dataset.glob('*/*.png'))}


In [ ]:
for class_name, indices in estimator.class_indices.items():
    print(f"Clase: {class_names}, Número de muestras: {len(indices)}")


Clase: ['cat', 'cat_c_defocus_blur'], Número de muestras: 1000
Clase: ['cat', 'cat_c_defocus_blur'], Número de muestras: 1000


In [ ]:
# Muestras mas similares

def show_most_confused(source_intent, target_intent, estimator, class_names, filenames_dict):
    print(f"{class_names[source_intent]} <> {class_names[target_intent]}")
    
    # Obtener las probabilidades de la clase objetivo
    probabilities = [sample.sample_probability_norm[target_intent] for sample in estimator.similarity_arrays[source_intent].values()]
    
    # Ordenar los índices según las probabilidades
    closest_to_second = np.argsort(probabilities)[::-1][:10]
    dataset_indices = estimator.class_indices[source_intent][closest_to_second]
    print(dataset_indices.shape)

    # Verificar que los índices estén dentro de los límites del dataset
    valid_indices = [int(di) for di in dataset_indices if int(di) < len(filenames_dict)]
    
    for di in valid_indices:
        try:
            filename = filenames_dict[int(di)]
            value = probabilities[closest_to_second[np.where(dataset_indices == di)[0][0]]]
            print(f"\tNombre del archivo: {filename}, Clase: {class_names[fully_labels[int(di)]]}, Similitud: {value:.4f}")
        except IndexError:
            print(f"\tIndexError: Index {int(di)} is out of bounds for the dataset.")

# Suponiendo que `pairs` contiene los índices de las clases en el formato [(0, 1)]
pairs = [(0, 1)]

first_pair = pairs[0]    
show_most_confused(first_pair[0], first_pair[1], estimator, class_names, filenames_dict)
show_most_confused(first_pair[1], first_pair[0], estimator, class_names, filenames_dict)
for class_name, indices in estimator.class_indices.items():
    print(f"Clase: {class_names}, Número de muestras: {len(indices)}")


cat <> cat_c_defocus_blur
(10,)
	Nombre del archivo: house_cat_s_000333.png, Clase: cat, Similitud: 10000.0000
	Nombre del archivo: domestic_cat_s_001272.png, Clase: cat, Similitud: 10000.0000
	Nombre del archivo: tabby_cat_s_001455.png, Clase: cat, Similitud: 10000.0000
	Nombre del archivo: cat_s_002262.png, Clase: cat, Similitud: 10000.0000
	Nombre del archivo: domestic_cat_s_000227.png, Clase: cat, Similitud: 10000.0000
	Nombre del archivo: tabby_s_000140.png, Clase: cat, Similitud: 10000.0000
	Nombre del archivo: cat_s_000411.png, Clase: cat, Similitud: 10000.0000
	Nombre del archivo: tabby_cat_s_002421.png, Clase: cat, Similitud: 10000.0000
	Nombre del archivo: tabby_cat_s_002349.png, Clase: cat, Similitud: 10000.0000
	Nombre del archivo: tabby_s_000459.png, Clase: cat, Similitud: 10000.0000
cat_c_defocus_blur <> cat
(10,)
	Nombre del archivo: 22643.png, Clase: cat_c_defocus_blur, Similitud: 6000.0000
	Nombre del archivo: 44617.png, Clase: cat_c_defocus_blur, Similitud: 6000.0000


In [ ]:
# Verificar el tamaño del dataset
print(f"Total de muestras en el dataset: {len(filenames_dict)}")

# Verificar el número de muestras en cada clase
for class_name, indices in estimator.class_indices.items():
    print(f"Clase: {class_name}, Número de muestras: {len(indices)}")

# Verificar el número de muestras en similarity_arrays
for class_name, samples in estimator.similarity_arrays.items():
    print(f"Clase: {class_name}, Número de muestras en similarity_arrays: {len(samples)}")

# Verificar que las probabilidades estén siendo calculadas para todas las muestras
for source_intent in range(len(class_names)):
    for target_intent in range(len(class_names)):
        probabilities = [sample.sample_probability_norm[target_intent] for sample in estimator.similarity_arrays[source_intent].values()]
        print(f"Clase fuente: {class_names[source_intent]}, Clase objetivo: {class_names[target_intent]}, Número de probabilidades calculadas: {len(probabilities)}")

# Verificar los índices seleccionados para cada clase
for source_intent in range(len(class_names)):
    probabilities = [sample.sample_probability_norm[target_intent] for sample in estimator.similarity_arrays[source_intent].values()]
    closest_to_second = np.argsort(probabilities)[::-1][:10]
    dataset_indices = estimator.class_indices[source_intent][closest_to_second]
    print(f"Clase fuente: {class_names[source_intent]}, Índices seleccionados: {dataset_indices}")


Total de muestras en el dataset: 10000
Clase: 0, Número de muestras: 1000
Clase: 1, Número de muestras: 1000
Clase: 0, Número de muestras en similarity_arrays: 1000
Clase: 1, Número de muestras en similarity_arrays: 1000
Clase fuente: cat, Clase objetivo: cat, Número de probabilidades calculadas: 1000
Clase fuente: cat, Clase objetivo: cat_c_defocus_blur, Número de probabilidades calculadas: 1000
Clase fuente: cat_c_defocus_blur, Clase objetivo: cat, Número de probabilidades calculadas: 1000
Clase fuente: cat_c_defocus_blur, Clase objetivo: cat_c_defocus_blur, Número de probabilidades calculadas: 1000
Clase fuente: cat, Índices seleccionados: [2216 1206 3278  662  894 3788  271 3598 3574 3921]
Clase fuente: cat_c_defocus_blur, Índices seleccionados: [5940 5808 8160 7021 9055 7988 6733 9521 5465 5883]


In [ ]:
# Muestras menos similares 

def show_least_confused(source_intent, target_intent, estimator, class_names, filenames_dict):
    print(f"{class_names[source_intent]} <> {class_names[target_intent]}")
    
    # Obtener las probabilidades de la clase objetivo
    probabilities = [sample.sample_probability_norm[target_intent] for sample in estimator.similarity_arrays[source_intent].values()]
    
    # Ordenar los índices según las probabilidades (de menor a mayor)
    least_similar = np.argsort(probabilities)[:10]
    dataset_indices = estimator.class_indices[source_intent][least_similar]
    
    print(dataset_indices.shape)
    # Verificar que los índices estén dentro de los límites del dataset
    valid_indices = [int(di) for di in dataset_indices if int(di) < len(filenames_dict)]
    
    for di in valid_indices:
        try:
            filename = filenames_dict[int(di)]
            value = probabilities[least_similar[np.where(dataset_indices == di)[0][0]]]
            print(f"\tNombre del archivo: {filename}, Clase: {class_names[fully_labels[int(di)]]}, Similitud: {value:.4f}")
        except IndexError:
            print(f"\tIndexError: Index {int(di)} is out of bounds for the dataset.")

# Suponiendo que `pairs` contiene los índices de las clases en el formato [(0, 1)]
pairs = [(0, 1)]

first_pair = pairs[0]    
show_least_confused(first_pair[0], first_pair[1], estimator, class_names, filenames_dict)
show_least_confused(first_pair[1], first_pair[0], estimator, class_names, filenames_dict)


cat <> cat_c_defocus_blur
(10,)
	Nombre del archivo: true_cat_s_001802.png, Clase: cat, Similitud: 0.0000
	Nombre del archivo: mouser_s_000441.png, Clase: cat, Similitud: 0.0000
	Nombre del archivo: tabby_cat_s_000227.png, Clase: cat, Similitud: 0.0000
	Nombre del archivo: domestic_cat_s_001919.png, Clase: cat, Similitud: 0.0000
	Nombre del archivo: true_cat_s_000954.png, Clase: cat, Similitud: 0.0000
	Nombre del archivo: tomcat_s_002230.png, Clase: cat, Similitud: 0.0000
	Nombre del archivo: tabby_s_000657.png, Clase: cat, Similitud: 0.0000
	Nombre del archivo: felis_domesticus_s_000184.png, Clase: cat, Similitud: 0.0000
	Nombre del archivo: felis_catus_s_001396.png, Clase: cat, Similitud: 0.0000
	Nombre del archivo: felis_domesticus_s_000673.png, Clase: cat, Similitud: 0.0000
cat_c_defocus_blur <> cat
(10,)
	Nombre del archivo: 28251.png, Clase: cat_c_defocus_blur, Similitud: 0.0000
	Nombre del archivo: 06530.png, Clase: cat_c_defocus_blur, Similitud: 0.0000
	Nombre del archivo: 4196

In [ ]:
# Comparación de muestras entre par de clases (estimator.P)

# Obtener todos los nombres de archivo en el dataset
filenames = [str(f) for f in directorio_dataset.glob('*/*.png')]

# Crear un mapeo de índices a nombres de archivo (solo el nombre de archivo, sin la ruta)
filenames_dict = {i: f.name for i, f in enumerate(directorio_dataset.glob('*/*.png'))}

def show_least_similar_using_P(source_intent, target_intent, estimator, class_names, filenames_dict):
    print(f"{class_names[source_intent]} <> {class_names[target_intent]}")
    
    # Obtener la matriz de similitud de las muestras entre las clases
    P = estimator.P[(source_intent, target_intent)]
    
    # Ordenar los índices según las similitudes (de menor a mayor)
    least_similar_indices = np.argsort(P, axis=None)[:10]
    
    # Convertir los índices planos a índices 2D
    least_similar_indices_2d = np.unravel_index(least_similar_indices, P.shape)
    print(P.shape)
    
    # Obtener los índices de las muestras en el dataset
    dataset_indices = [estimator.class_indices[source_intent][idx] for idx in least_similar_indices_2d[0]]
    
    # Verificar que los índices estén dentro de los límites del dataset
    valid_indices = [int(di) for di in dataset_indices if int(di) < len(filenames_dict)]
    
    for di in valid_indices:
        try:
            filename = filenames_dict[int(di)]
            value = P[least_similar_indices_2d[0][np.where(np.array(dataset_indices) == di)[0][0]], 
                      least_similar_indices_2d[1][np.where(np.array(dataset_indices) == di)[0][0]]]
            print(f"\tNombre del archivo: {filename}, Clase: {class_names[fully_labels[int(di)]]}, Similitud: {value:.4f}")
        except IndexError:
            print(f"\tIndexError: Index {int(di)} is out of bounds for the dataset.")

# Suponiendo que `pairs` contiene los índices de las clases en el formato [(0, 1)]
pairs = [(0, 1)]

first_pair = pairs[0]


# Mostrar los menos similares usando la matriz de pares (P)
show_least_similar_using_P(first_pair[0], first_pair[1], estimator, class_names, filenames_dict)
show_least_similar_using_P(first_pair[1], first_pair[0], estimator, class_names, filenames_dict)
for class_name, indices in estimator.class_indices.items():
    print(f"Clase: {class_name}, Número de muestras: {len(indices)}")

cat <> cat_c_defocus_blur
(1000, 1000)
	Nombre del archivo: tomcat_s_001532.png, Clase: cat, Similitud: 0.0559
	Nombre del archivo: tomcat_s_001532.png, Clase: cat, Similitud: 0.0559
	Nombre del archivo: felis_catus_s_000312.png, Clase: cat, Similitud: 0.0842
	Nombre del archivo: felis_catus_s_000312.png, Clase: cat, Similitud: 0.0842
	Nombre del archivo: domestic_cat_s_001530.png, Clase: cat, Similitud: 0.0956
	Nombre del archivo: domestic_cat_s_001530.png, Clase: cat, Similitud: 0.0956
	Nombre del archivo: cat_s_002148.png, Clase: cat, Similitud: 0.0975
	Nombre del archivo: tomcat_s_001532.png, Clase: cat, Similitud: 0.0559
	Nombre del archivo: cat_s_002148.png, Clase: cat, Similitud: 0.0975
	Nombre del archivo: felis_catus_s_000317.png, Clase: cat, Similitud: 0.1031
cat_c_defocus_blur <> cat
(1000, 1000)
	Nombre del archivo: 19246.png, Clase: cat_c_defocus_blur, Similitud: 0.0559
	Nombre del archivo: 39246.png, Clase: cat_c_defocus_blur, Similitud: 0.0616
	Nombre del archivo: 19246.

In [ ]:
# Verificar el tamaño del dataset
print(f"Total de muestras en el dataset: {len(filenames_dict)}")

# Verificar el número de muestras en cada clase
for class_name, indices in estimator.class_indices.items():
    print(f"Clase: {class_name}, Número de muestras: {len(indices)}")

# Verificar el número de muestras en similarity_arrays
for class_name, samples in estimator.similarity_arrays.items():
    print(f"Clase: {class_name}, Número de muestras en similarity_arrays: {len(samples)}")

# Verificar que las probabilidades estén siendo calculadas para todas las muestras
for source_intent in range(len(class_names)):
    for target_intent in range(len(class_names)):
        probabilities = [sample.sample_probability_norm[target_intent] for sample in estimator.similarity_arrays[source_intent].values()]
        print(f"Clase fuente: {class_names[source_intent]}, Clase objetivo: {class_names[target_intent]}, Número de probabilidades calculadas: {len(probabilities)}")

# Verificar los índices seleccionados para cada clase
for source_intent in range(len(class_names)):
    probabilities = [sample.sample_probability_norm[target_intent] for sample in estimator.similarity_arrays[source_intent].values()]
    closest_to_second = np.argsort(probabilities)[::-1][:10]
    dataset_indices = estimator.class_indices[source_intent][closest_to_second]
    print(f"Clase fuente: {class_names[source_intent]}, Índices seleccionados: {dataset_indices}")

Total de muestras en el dataset: 10000
Clase: 0, Número de muestras: 1000
Clase: 1, Número de muestras: 1000
Clase: 0, Número de muestras en similarity_arrays: 1000
Clase: 1, Número de muestras en similarity_arrays: 1000
Clase fuente: cat, Clase objetivo: cat, Número de probabilidades calculadas: 1000
Clase fuente: cat, Clase objetivo: cat_c_defocus_blur, Número de probabilidades calculadas: 1000
Clase fuente: cat_c_defocus_blur, Clase objetivo: cat, Número de probabilidades calculadas: 1000
Clase fuente: cat_c_defocus_blur, Clase objetivo: cat_c_defocus_blur, Número de probabilidades calculadas: 1000
Clase fuente: cat, Índices seleccionados: [2216 1206 3278  662  894 3788  271 3598 3574 3921]
Clase fuente: cat_c_defocus_blur, Índices seleccionados: [5940 5808 8160 7021 9055 7988 6733 9521 5465 5883]


In [ ]:

# Obtener los índices de las muestras para cada clase
class_indices = estimator.class_indices
indices_clase_cat = class_indices[0]
indices_clase_defocus_blur = class_indices[1]

# Imprimir las primeras diez muestras de la clase cat
print(f"Clase: {class_names[0]}")
for i in range(10):
    idx = indices_clase_cat[i]
    print(f"\tÍndice: {idx}, Nombre del archivo: {filenames_dict[idx]}")

# Imprimir las primeras diez muestras de la clase cat_c_defocus_blur
print(f"\nClase: {class_names[1]}")
for i in range(10):
    idx = indices_clase_defocus_blur[i] # Ajuste de índices para la clase cat_c_defocus_blur
    print(f"\tÍndice: {idx}, Nombre del archivo: {filenames_dict[idx]}")




Clase: cat
	Índice: 3192, Nombre del archivo: tabby_cat_s_001083.png
	Índice: 3622, Nombre del archivo: tabby_cat_s_002517.png
	Índice: 361, Nombre del archivo: cat_s_000801.png
	Índice: 874, Nombre del archivo: domestic_cat_s_000176.png
	Índice: 3989, Nombre del archivo: tabby_s_000615.png
	Índice: 3625, Nombre del archivo: tabby_cat_s_002523.png
	Índice: 3810, Nombre del archivo: tabby_s_000197.png
	Índice: 194, Nombre del archivo: cat_s_000102.png
	Índice: 862, Nombre del archivo: domestic_cat_s_000129.png
	Índice: 2907, Nombre del archivo: tabby_cat_s_000204.png

Clase: cat_c_defocus_blur
	Índice: 9468, Nombre del archivo: 44765.png
	Índice: 8491, Nombre del archivo: 34965.png
	Índice: 5971, Nombre del archivo: 09665.png
	Índice: 8424, Nombre del archivo: 34404.png
	Índice: 5967, Nombre del archivo: 09578.png
	Índice: 8370, Nombre del archivo: 33752.png
	Índice: 5181, Nombre del archivo: 01848.png
	Índice: 6932, Nombre del archivo: 19218.png
	Índice: 8795, Nombre del archivo: 37952

In [ ]:

# Obtener los índices de las muestras para cada clase
class_indices = estimator.class_indices
indices_clase_cat = class_indices[0]
indices_clase_defocus_blur = class_indices[1]

# Imprimir las primeras diez muestras de la clase cat
print(f"Clase: {class_names[0]}")
for i in range({list(filenames_dict.items())[10:]}):
    idx = indices_clase_cat[i]
    print(f"\tÍndice: {idx}, Nombre del archivo: {filenames_dict[idx]}")

# Imprimir las primeras diez muestras de la clase cat_c_defocus_blur
print(f"\nClase: {class_names[1]}")
for i in range({list(filenames_dict.items())[-10:]}):
    idx = indices_clase_defocus_blur[i] # Ajuste de índices para la clase cat_c_defocus_blur
    print(f"\tÍndice: {idx}, Nombre del archivo: {filenames_dict[idx]}")

Clase: cat


TypeError: 'dict_items' object is not subscriptable

In [ ]:
# Verificar el contenido de filenames_dict
print(f"Total archivos en filenames_dict: {len(filenames_dict)}")
print(f"Ultimos 10 archivos en filenames_dict: {list(filenames_dict.items())[-10:]}")


Total archivos en filenames_dict: 10000
Ultimos 10 archivos en filenames_dict: [(9990, '49947.png'), (9991, '49949.png'), (9992, '49950.png'), (9993, '49961.png'), (9994, '49967.png'), (9995, '49968.png'), (9996, '49979.png'), (9997, '49992.png'), (9998, '49994.png'), (9999, '49996.png')]


In [ ]:
# Muestras menos similares usando estimator.M

# Obtener todos los nombres de archivo en el dataset
filenames = [str(f) for f in directorio_dataset.glob('*/*.png')]

# Crear un mapeo de índices a nombres de archivo (solo el nombre de archivo, sin la ruta)
filenames_dict = {i: f.name for i, f in enumerate(directorio_dataset.glob('*/*.png'))}

def show_least_similar_using_M(source_intent, target_intent, estimator, class_names, filenames_dict):
    print(f"{class_names[source_intent]} <> {class_names[target_intent]}")
    
    # Obtener la matriz de similitud de todas las muestras
    M = estimator.M
    
    # Ordenar los índices según las similitudes (de menor a mayor)
    least_similar_indices = np.argsort(M, axis=None)[:10]
    
    # Convertir los índices planos a índices 2D
    least_similar_indices_2d = np.unravel_index(least_similar_indices, M.shape)
    
    # Obtener los índices de las muestras en el dataset
    valid_dataset_indices = []
    for idx in least_similar_indices_2d[0]:
        if idx < len(estimator.class_indices[source_intent]):
            valid_dataset_indices.append(estimator.class_indices[source_intent][idx])
    
    # Verificar que los índices estén dentro de los límites del dataset
    valid_indices = [int(di) for di in valid_dataset_indices if int(di) < len(filenames_dict)]
    
    for di in valid_indices:
        try:
            filename = filenames_dict[int(di)]
            value = M[least_similar_indices_2d[0][np.where(np.array(valid_dataset_indices) == di)[0][0]], 
                      least_similar_indices_2d[1][np.where(np.array(valid_dataset_indices) == di)[0][0]]]
            print(f"\tNombre del archivo: {filename}, Clase: {class_names[fully_labels[int(di)]]}, Similitud: {value:.4f}")
        except IndexError:
            print(f"\tIndexError: Index {int(di)} is out of bounds for the dataset.")

# Suponiendo que `pairs` contiene los índices de las clases en el formato [(0, 1)]
pairs = [(0, 1)]

first_pair = pairs[0]

# Mostrar los menos similares usando la matriz de todas las muestras (M)
show_least_similar_using_M(first_pair[0], first_pair[1], estimator, class_names, filenames_dict)
show_least_similar_using_M(first_pair[1], first_pair[0], estimator, class_names, filenames_dict)


cat <> cat_c_defocus_blur
	Nombre del archivo: alley_cat_s_000731.png, Clase: cat, Similitud: 0.5786
	Nombre del archivo: alley_cat_s_000862.png, Clase: cat, Similitud: 0.5786
	Nombre del archivo: alley_cat_s_000428.png, Clase: cat, Similitud: 0.5813
	Nombre del archivo: alley_cat_s_000731.png, Clase: cat, Similitud: 0.5786
	Nombre del archivo: alley_cat_s_000428.png, Clase: cat, Similitud: 0.5813
cat_c_defocus_blur <> cat
	Nombre del archivo: alley_cat_s_001138.png, Clase: cat, Similitud: 0.5786
	Nombre del archivo: alley_cat_s_002076.png, Clase: cat, Similitud: 0.5786
	Nombre del archivo: alley_cat_s_001876.png, Clase: cat, Similitud: 0.5813
	Nombre del archivo: alley_cat_s_001138.png, Clase: cat, Similitud: 0.5786
	Nombre del archivo: alley_cat_s_001876.png, Clase: cat, Similitud: 0.5813


In [ ]:
# Muestras menos similares usando estimator.M

# Obtener todos los nombres de archivo en el dataset
filenames = [str(f) for f in directorio_dataset.glob('*/*.png')]

# Crear un mapeo de índices a nombres de archivo (solo el nombre de archivo, sin la ruta)
filenames_dict = {i: f.name for i, f in enumerate(directorio_dataset.glob('*/*.png'))}

def show_least_similar_using_M(source_intent, target_intent, estimator, class_names, filenames_dict):
    print(f"{class_names[source_intent]} <> {class_names[target_intent]}")
    
    # Obtener la matriz de similitud de todas las muestras
    M = estimator.M
    
    # Ordenar los índices según las similitudes (de menor a mayor)
    least_similar_indices = np.argsort(M)[:10]
    
    # Obtener los índices de las muestras en el dataset
    dataset_indices = estimator.class_indices[source_intent][least_similar_indices]
    
    # Verificar que los índices estén dentro de los límites del dataset
    valid_indices = [int(di) for di in dataset_indices if int(di) < len(filenames_dict)]
    
    for di in valid_indices:
        try:
            filename = filenames_dict[int(di)]
            value = M[least_similar_indices[np.where(dataset_indices == di)[0][0]]]
            print(f"\tNombre del archivo: {filename}, Clase: {class_names[fully_labels[int(di)]]}, Similitud: {value:.4f}")
        except IndexError:
            print(f"\tIndexError: Index {int(di)} is out of bounds for the dataset.")

# Suponiendo que `pairs` contiene los índices de las clases en el formato [(0, 1)]
pairs = [(0, 1)]

first_pair = pairs[0]

# Mostrar los menos similares usando la matriz de todas las muestras (M)
show_least_similar_using_M(first_pair[0], first_pair[1], estimator, class_names, filenames_dict)
show_least_similar_using_M(first_pair[1], first_pair[0], estimator, class_names, filenames_dict)


cat <> cat_c_defocus_blur


IndexError: index 118 is out of bounds for axis 0 with size 50